In [1]:
import os
import sys

from matplotlib import pyplot as plt
from matplotlib import patches
from matplotlib import colors
import pandas as pd
import IPython.display as ipd
import glob
import numpy as np
# import muspy
# import pypianoroll
import torch
# from util.play_midi import play_midi
!pip install miditok symusic
from miditok import REMI, TokenizerConfig
from miditok.pytorch_data import DatasetTok, DataCollator
from pathlib import Path
from symusic import Score
from torch.utils.data import DataLoader


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 55.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
config = TokenizerConfig()
tokenizer = REMI()
midi_files = list(Path("/content/drive/MyDrive/uni/Project/code/audio/midi_songs").glob("**/*.mid"))
print(f"number of midi_files = {len(midi_files)}")
print(f"number of vocab = {len(tokenizer.vocab)}")

number of midi_files = 93
number of vocab = 282


In [4]:
print(len(midi_files))
corrupt_files = []
for file in midi_files: # removing corrupted files/ files that cant be fully understood
    try:
        Score(file)
    except:
        corrupt_files.append(midi_files.pop(file))
print(len(midi_files))
for file in midi_files: # on windows this has to be run 2 times, i have no idea why.... very wierd
    try:
        Score(file)
    except:
        corrupt_files.append(midi_files.pop(file))
print(len(midi_files))

93
93
93


In [19]:
song = Score(midi_files[-18])
tokenized_song = tokenizer.midi_to_tokens(song)

song1 =tokenizer.tokens_to_midi(tokenized_song)
ids = tokenizer._tokens_to_ids(tokenized_song[-1])
len(tokenizer._ids_to_tokens(ids))
ids

[4,
 189,
 29,
 110,
 152,
 190,
 36,
 113,
 151,
 39,
 110,
 151,
 44,
 115,
 151,
 217,
 32,
 114,
 158,
 218,
 43,
 114,
 157,
 39,
 116,
 158,
 48,
 111,
 157,
 4,
 4,
 189,
 34,
 110,
 163,
 44,
 111,
 171,
 41,
 113,
 171,
 191,
 49,
 106,
 170]

In [ ]:
# tokenizer.learn_bpe(vocab_size=500, files_paths=midi_files)


In [6]:
from transformers import DataCollatorForLanguageModeling

dataset = DatasetTok( # seq_len = start + seq_len + end
    files_paths=midi_files,
    min_seq_len=50, # 52
    max_seq_len=254, # 514
    tokenizer=tokenizer
)

collator = DataCollator(
    tokenizer["PAD_None"], tokenizer["BOS_None"], tokenizer["EOS_None"], copy_inputs_as_labels=True
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

data_loader = DataLoader(dataset=dataset, batch_size=256, collate_fn=collator,)

Loading data: /content/drive/MyDrive/uni/Project/code/audio/midi_songs: 100%|██████████| 93/93 [00:00<00:00, 271.15it/s]


In [ ]:
for batch in data_loader:
    x, y, z = **batch
    break
y = **x

{'input_ids': tensor([[  1,   4, 189,  ...,   0,   0,   0],
         [  1,   4,   4,  ..., 132,   4,   2],
         [  1, 189,  15,  ...,   0,   0,   0],
         ...,
         [  1,   4,   4,  ...,   0,   0,   0],
         [  1,   4,   4,  ...,   0,   0,   0],
         [  1,   4, 205,  ...,   0,   0,   0]]),
 'labels': tensor([[   1,    4,  189,  ..., -100, -100, -100],
         [   1,    4,    4,  ...,  132,    4,    2],
         [   1,  189,   15,  ..., -100, -100, -100],
         ...,
         [   1,    4,    4,  ..., -100, -100, -100],
         [   1,    4,    4,  ..., -100, -100, -100],
         [   1,    4,  205,  ..., -100, -100, -100]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]], dtype=torch.int32)}

In [14]:
y = tokenizer._ids_to_tokens(x["input_ids"][0].tolist())


NameError: name 'x' is not defined

In [7]:
from transformers import AutoConfig, GPT2LMHeadModel

device = "cuda" if torch.cuda.is_available() else "cpu"



In [22]:
context_length = 256
n_layer = 6
n_head = 8
n_emb = 256 # 512

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_positions=context_length,
    n_layer=n_layer,
    n_head=n_head,
    pad_token_id=tokenizer["PAD_None"],
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
    n_embd=n_emb
)


In [23]:
model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size} parameters")
model

GPT-2 size: 4876800 parameters


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(282, 256)
    (wpe): Embedding(256, 256)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=256, out_features=282, bias=False)
)

In [29]:
from transformers.tokenization_utils_base import BatchEncoding

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

epochs = 100
for epoch in range(epochs):
    total_loss = []
    model.train()
    for batch in data_loader:
        model.zero_grad()
        # full = torch.tensor(batch).to(device)
        full_inputs = BatchEncoding()
        full_inputs.update(batch)
        inputs = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)
        mask = batch["attention_mask"].to(device)
        outputs = model(**full_inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss.append(loss)
    print(f"Epock = {epoch+1} \nloss = {loss}")


Epock = 1 
loss = 4.080315113067627
Epock = 2 
loss = 4.88307523727417
Epock = 3 
loss = 4.390296936035156
Epock = 4 
loss = 4.267981052398682
Epock = 5 
loss = 4.30527925491333
Epock = 6 
loss = 4.03278923034668
Epock = 7 
loss = 3.999056577682495
Epock = 8 
loss = 4.026047706604004
Epock = 9 
loss = 3.9749960899353027
Epock = 10 
loss = 3.8980295658111572
Epock = 11 
loss = 3.866860866546631
Epock = 12 
loss = 3.834446907043457
Epock = 13 
loss = 3.787245035171509
Epock = 14 
loss = 3.7355384826660156
Epock = 15 
loss = 3.699366569519043
Epock = 16 
loss = 3.6793198585510254
Epock = 17 
loss = 3.6651341915130615
Epock = 18 
loss = 3.6434450149536133
Epock = 19 
loss = 3.61387038230896
Epock = 20 
loss = 3.588630437850952
Epock = 21 
loss = 3.5734989643096924
Epock = 22 
loss = 3.558274269104004
Epock = 23 
loss = 3.543020486831665
Epock = 24 
loss = 3.530461311340332
Epock = 25 
loss = 3.512260675430298
Epock = 26 
loss = 3.4939494132995605
Epock = 27 
loss = 3.4759960174560547
Epock

In [ ]:
model.eval()

with torch.no_grad():
    for batch in data_loader:
        full_inputs = BatchEncoding()
        full_inputs.update(batch)
        labels = batch["labels"].to(device)
        outputs = model(**full_inputs)
        break
print(outputs)

In [30]:
output_tokens = torch.tensor([[0]*256]*256)
for i in range(len(outputs["logits"])):
    output_tokens[i] = torch.argmax(outputs["logits"][i], dim=-1)

print(output_tokens[0])

tensor([  4, 189,  35, 108, 127,  51, 108, 127,  53, 108, 127,  53, 108, 127,
         53,  31, 108, 127, 205, 108, 127,  54, 108, 127, 205, 108, 127,  53,
        108, 127,  53, 108, 127,   4, 108, 127, 205, 189,  31, 108, 127,  53,
        108, 127,  53, 108, 127,  53, 108, 127,  53, 108, 127, 205,  32, 108,
        127,  53, 108, 127, 205, 108, 127,  53, 108, 127,  53, 108, 127,   4,
        108, 127,  53, 108, 127, 205,  31, 108, 127,  53, 108, 127,  53, 108,
        127,  53, 108, 127,   4, 108, 127,   4, 108, 127,  53, 108, 127,  53,
         31, 127, 108, 108, 127, 127, 108, 127, 108, 127, 108, 108, 127, 108,
        127, 108, 108, 108, 108, 127, 127, 108,  53, 108, 108, 127, 108, 108,
        108, 108, 108, 108, 108, 127, 108, 108, 108, 127, 127, 127, 108, 127,
        127, 108, 127, 127, 108, 108, 108, 127, 127, 108, 108, 127, 127, 127,
        108, 108, 127, 127, 108, 108, 127, 108, 127, 108, 127, 108, 108, 127,
        127, 127, 108, 108,  53, 108, 127, 127, 108, 108, 108, 1

In [15]:
# midi = tokenizer([output_tokens[0]])

In [31]:
tokenizer._ids_to_tokens(output_tokens[0].tolist())

['Bar_None',
 'Position_0',
 'Pitch_51',
 'Velocity_63',
 'Duration_0.3.8',
 'Pitch_67',
 'Velocity_63',
 'Duration_0.3.8',
 'Pitch_69',
 'Velocity_63',
 'Duration_0.3.8',
 'Pitch_69',
 'Velocity_63',
 'Duration_0.3.8',
 'Pitch_69',
 'Pitch_47',
 'Velocity_63',
 'Duration_0.3.8',
 'Position_16',
 'Velocity_63',
 'Duration_0.3.8',
 'Pitch_70',
 'Velocity_63',
 'Duration_0.3.8',
 'Position_16',
 'Velocity_63',
 'Duration_0.3.8',
 'Pitch_69',
 'Velocity_63',
 'Duration_0.3.8',
 'Pitch_69',
 'Velocity_63',
 'Duration_0.3.8',
 'Bar_None',
 'Velocity_63',
 'Duration_0.3.8',
 'Position_16',
 'Position_0',
 'Pitch_47',
 'Velocity_63',
 'Duration_0.3.8',
 'Pitch_69',
 'Velocity_63',
 'Duration_0.3.8',
 'Pitch_69',
 'Velocity_63',
 'Duration_0.3.8',
 'Pitch_69',
 'Velocity_63',
 'Duration_0.3.8',
 'Pitch_69',
 'Velocity_63',
 'Duration_0.3.8',
 'Position_16',
 'Pitch_48',
 'Velocity_63',
 'Duration_0.3.8',
 'Pitch_69',
 'Velocity_63',
 'Duration_0.3.8',
 'Position_16',
 'Velocity_63',
 'Duration

In [32]:
torch.save(model.state_dict(), "remi_gpt2.pt")